In [1]:
import numpy as np
import queue
from skimage import io

In [45]:
GTS_IND_LIST  = [3,5,11,13,17,21,23,26,28,30,32,34,37] 

MAIN_PATH = "/Users/ekaterina/Documents/Semantic Segmentation/ISPRS_semantic_labeling_Vaihingen"

GTS_FORMAT = "/gts_for_participants/top_mosaic_09cm_area{}.tif"
COV_RES = "/Output/class_dependence.txt"

categories = np.zeros((6, 3), dtype = 'uint8')
categories[0]  = [255, 255, 255]
categories[1]  = [0, 0, 255]
categories[2]  = [0, 255, 255]
categories[3]  = [0, 255, 0]
categories[4]  = [255, 255, 0]
categories[5]  = [255, 0, 0]

In [3]:
def get_gts(num):
    label_path = MAIN_PATH + "/gts_for_participants/top_mosaic_09cm_area{}.tif". format(num)
    res = io.imread(label_path)
    return res

In [4]:
def convert_color_to_labels(color_map):
    label_map = np.zeros((color_map.shape[0],color_map.shape[1]), dtype = 'uint8')
    for i in range(color_map.shape[0]):
        for j in range (color_map.shape[1]):
            pics = color_map[i][j][:]
            for k in range(6):
                if np.array_equal(pics, categories[k, :]):
                    label_map[i][j] = k
                    break
    return label_map

In [13]:
def bfs(img):
    bfs_queue= queue.Queue()
    bfs_queue.put((0,0))
    visited = np.zeros((img.shape[0],img.shape[1],1), dtype = 'uint64')
    while not bfs_queue.empty():
        x,y = bfs_queue.get()
        if (x>0 and not visited[x-1][y]):
            if (img[x,y] != img[x-1,y]):
                neighbours[img[x,y], img[x-1,y]] += 1
            visited[x-1][y] = 1
            bfs_queue.put((x-1,y)) 
        if (y>0 and not visited[x][y-1]):
            if (img[x,y-1] != img[x,y]):
                neighbours[img[x,y], img[x,y-1]] += 1
            visited[x][y-1] = 1
            bfs_queue.put((x,y-1)) 
        if ((x!= (img.shape[0] - 1)) and not visited[x+1][y]):
            if (img[x+1,y] != img[x,y]):
                neighbours[img[x,y], img[x+1,y]] += 1
            visited[x+1][y] = 1
            bfs_queue.put((x+1,y)) 
        if ((y!= (img.shape[1] - 1)) and not visited[x][y+1]):
            if (img[x,y+1] != img[x,y]):
                neighbours[img[x,y], img[x,y+1]] += 1
            visited[x][y+1] = 1
            bfs_queue.put((x,y+1))
    return 0

In [14]:
neighbours = np.zeros((6, 6), dtype = 'uint64')
for i in GTS_IND_LIST:
    bfs(convert_color_to_labels(get_gts(i)))

In [46]:
neighbours = np.transpose(neighbours) + neighbours
res = np.divide(neighbours, np.sum(neighbours)/200)
np.savetxt(MAIN_PATH + COV_RES, res)

In [48]:
neighbours

array([[     0,  95771, 102850, 125273,  61150,   2881],
       [ 95771,      0,  93510,  39782,   1125,    176],
       [102850,  93510,      0, 190696,   1915,    723],
       [125273,  39782, 190696,      0,   2269,   3491],
       [ 61150,   1125,   1915,   2269,      0,      7],
       [  2881,    176,    723,   3491,      7,      0]], dtype=uint64)

In [49]:
res

array([[  0.    ,  13.2717,  14.2527,  17.36  ,   8.474 ,   0.3992],
       [ 13.2717,   0.    ,  12.9584,   5.5129,   0.1559,   0.0244],
       [ 14.2527,  12.9584,   0.    ,  26.4261,   0.2654,   0.1002],
       [ 17.36  ,   5.5129,  26.4261,   0.    ,   0.3144,   0.4838],
       [  8.474 ,   0.1559,   0.2654,   0.3144,   0.    ,   0.001 ],
       [  0.3992,   0.0244,   0.1002,   0.4838,   0.001 ,   0.    ]])